In [86]:
import jieba
import pandas as pd
from collections import defaultdict
import math
import operator

jieba.initialize()

In [143]:
train_df = pd.read_csv('datasets/train_dataset_pre_summary.csv', sep='|')

In [144]:
train_df.sample()

,Unnamed: 0,id,content,abstract,texts,summaries,pred_summary,metric,couverage
18917,18935,18917,【坐席】您好，很高兴为您服务？【客户】，喂，你好我，我问一下我，我这个网现在怎么这么慢了速度...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,"['您好，很高兴为您服务？', '，喂，你好我，我问一下我，我这个网现在怎么这么慢了速度，嗯...","[12, 30, 34]",嗯嗯，那您的意思是我们这边的信号有问题，北京北方驾校失效，嗯，我看到，嗯，非常感谢您向我们这...,0.434095,0.782609


In [146]:
train_df.groupby(pd.cut(train_df["metric"],5)).count()

,Unnamed: 0,id,content,abstract,texts,summaries,pred_summary,metric,couverage
metric,,,,,,,,,
"(-0.000773, 0.155]",18210,18210,18210,18210,18210,18210,18210,18210,18210
"(0.155, 0.309]",5583,5583,5583,5583,5583,5583,5583,5583,5583
"(0.309, 0.464]",653,653,653,653,653,653,653,653,653
"(0.464, 0.618]",467,467,467,467,467,467,467,467,467
"(0.618, 0.773]",88,88,88,88,88,88,88,88,88


In [198]:
train_df[train_df["abstract"].str.contains('1、')&train_df["abstract"].str.contains(':')].shape

(57, 9)

In [ ]:
我方查询用户号码状态**，用户现对此不认可，要求我方核实原因并回电请处理谢谢!

# 最简单的TF-IDF分类算法

In [87]:
def load_abstract():
    train_df_iter = pd.read_csv('datasets/train_dataset.csv', sep='|')
    results = []
    for i in train_df_iter['abstract'].tolist():

        target = jieba.cut(i, HMM=False)
        results.append([i for i in target])
    return results

def feature_select(list_words):
    # 总词频统计
    doc_frequency = defaultdict(int)
    for word_list in list_words:
        for i in word_list:
            doc_frequency[i] += 1

    # 计算每个词的TF值
    word_tf = {}  # 存储没个词的tf值
    for i in doc_frequency:
        word_tf[i] = doc_frequency[i] / sum(doc_frequency.values())

    # 计算每个词的IDF值
    doc_num = len(list_words)
    word_idf = {}  # 存储每个词的idf值
    word_doc = defaultdict(int)  # 存储包含该词的文档数
    for i in doc_frequency:
        for j in list_words:
            if i in j:
                word_doc[i] += 1
    for i in doc_frequency:
        word_idf[i] = math.log(doc_num / (word_doc[i] + 1))

    # 计算每个词的TF*IDF的值
    word_tf_idf = {}
    for i in doc_frequency:
        word_tf_idf[i] = word_tf[i] * word_idf[i]

    # 对字典按值由大到小排序
    dict_feature_select = sorted(word_tf_idf.items(), key=operator.itemgetter(1), reverse=True)
    return dict_feature_select


data_list = load_abstract()  # 加载数据
features = feature_select(data_list)  # 所有词的TF-IDF值
print(len(features))

11594


In [114]:
with open('datasets/tf_idf.text', 'w', encoding='UTF-8') as writer:
    writer.writelines('\n'.join([f'{i[0]} {i[1]*1000}' for i in features]))

In [134]:
import jieba.analyse
import pandas as pd
jieba.analyse.set_idf_path("./datasets/tf_idf.text")

def find_key_words(row):
    return jieba.analyse.textrank(row.encode('utf-8'), topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))


# def load_abstract():
#     train_df_iter = pd.read_csv('datasets/train_dataset_key.csv', sep='|')
#     train_df_iter['testxkeys'] = train_df_iter['abstract'].apply(find_key_words)
#     return train_df_iter


# train_df = load_abstract()  # 加载数据


In [126]:
for _,row in train_df.sample(n=5).iterrows():
    print(row['abstract'])
    print(row['reverst_keys'])
    print(row['keys'])

用户来电反映参与双12 活动，用户称只有一某某人50元2某某人10元另外3某某人没有到账，用户要求核实原因，请处理，谢谢开源花苑7楼2单元1902号
['开源', '花苑', '1902', '单元', '参与']
['用户', '某某人', '核实', '反映', '没有']
用户来电表示对当前欠费不认可9月-11月共计300.06元，称之前去了马家堡附件的营业厅销户，但是去了以后当时人员告知属于合作厅需要到大型的营业厅去办理销户，用户表示北京不好停车，后续一直就没有办理成功，现在用户对此不认可，表示自己只是一个用户，不清楚联通还有合作厅和营业厅，在自己看来都是联通的营业厅，要求我方给一个说法，请核实，谢谢！
['大型', '看来', '300.06', '停车', '马家堡']
['用户', '营业厅', '自己', '表示', '办理']
用户来电反映手机上网流量比上个月走的流量多，用户是腾讯王卡，我方按业务口径向用户解释腾讯40G免流量用户称上个月也是正常使用免流的流量也没有显示走这么多，对流量的使用情况不认可，用户要求我方核实原因，请处理，谢谢
['王卡', '免流', '40G', '上个月', '这么']
['用户', '流量', '我方', '使用', '正常']
用户再次来电反映手机号码无法正常使用短信的问题，我方按照笔记向用户解释，详见：TS********，用户称自己需要登录微信，对于封停时限不认可，请处理，谢谢
['微信', '封停', '笔记', '登录', '时限']
['用户', '自己', '我方', '使用', '正常']
客户您好！您反映的问题我们已经记录。您投诉的区域，已有基站建设计划，我们正在获取资源，因涉及在别人地方上加资源首先要得到当地居民（或业主）、物业的允许,目前方案正在制定，仍需一段时间才能开工建设，希望您能耐心等待。如果上述资源全部协调顺利，最长需要三个月才能完成建站，如果您有什么物业关系，能够加快相关资源协调力度，也希望您拨打10100和我们联系，我们自身也会加大协调力度，尽早解决您反映的问题。;投诉现象最近发生时间：一直信号不好;
['上加', '10100', '最长', '顺利', '加快']
['我们', '资源', '问题', '反映', '投诉']


In [130]:
train_df = train_df.rename(columns={'keys':'high_key'})

In [132]:
train_df.head()

,id,content,abstract,reverst_keys,high_key
0,0,【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把...,用户来电反映手机号卡收费问题，用户称现在怎么欠费167.65，我方经查询用户有10月欠费76...,"['167.65', '此号', '就别', '扣取', '没用']","[用户, 费用, 欠费, 我方, 使用]"
1,1,【客户】嗯，【坐席】，很高兴为您服务，【客户】喂，你好，你好，我想问一下，我没有去过这个吉林...,用户来电反映查询漫游地，显示有吉林，用户称没有去过附近，要求核实回复，请处理谢谢,"['吉林', '附近', '漫游', '去过', '谢谢']","[用户, 查询, 核实, 反映, 没有]"
2,2,【客户】嗯，【坐席】，您好，实习话务员为您服务，【客户】喂你好，然后我这某某人卡，然后有有两...,用户来电想要取消20元语音特惠包（200分钟另得100条短信）-立即生效，我方取消不成功，用...,"['另得', '特惠', '200', '想要', '立即']","[用户, 我方, 取消, 核实, 短信]"
3,3,【坐席】很高兴为您服务，您好？【客户】，唉，您好，那个，我这手机号上个月就是因为我每个月固定...,用户来电反映自己手机号码在7月份24.10元 8月份元 11月份85.90元 12月份300...,"['24.10', '85.90', '详细', '300', '11']","[用户, 月份, 我方, 认可, 流量]"
4,4,【坐席】您好，高兴为您服务，【客户】，唉，你好那个，我家里这个***不太好，【坐席】信号不好...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,"['谢谢', '以前', '宝贵', '持续', '这些']","[我们, 信号, 最近, 使用, 问题]"


In [136]:
train_df['textrank'] = train_df['abstract'].apply(find_key_words)

In [139]:
del train_df['tests_rank']

In [140]:
train_df.to_csv('datasets/train_dataset_key.csv', index=False, sep='|')

In [232]:
train_df = pd.read_csv('datasets/train_dataset_key.csv', sep='|')

In [212]:
cc = train_df.sample()
print(cc['abstract'].iloc[0])

用户对其号码套餐目前欠费有异议，称之前已预存费用。我方按套餐解释，其当时预存费用可抵扣优惠期内的前几个月的套餐费，预存款抵扣完，即需及时交费。用户不认可，称当时告知预存费用后，优惠期内不需交费、或每月只需交几十元。用户要求查询当时的办理过程。请处理，谢谢！


In [ ]:
相似的信息
信用额度：透支
中文转数字
带有'\t'得都删除


限速
骚扰
前期
信号
短信
基站
信用额度
流量
前期
减免

所在位置->基站
#rule base
奇怪的abstract的格式，
交了费用但是没有续费成功


In [234]:
train_df = train_df[~(train_df["abstract"].str.contains('\t\t')&train_df["abstract"].str.contains(':'))]

In [235]:
train_df = train_df[~(train_df["abstract"].str.contains('1、')&train_df["abstract"].str.contains(':'))]

In [254]:
def find_key_words(row):
    result = []
    for i in ['限速', '骚扰', '前期', '信号', '短信', '基站', '额度', '流量', '前期', '减免']:
        if i in row['abstract']:
            result.append(i)
    return result

In [255]:
train_df['own_key'] = train_df.apply(find_key_words,axis=1)

In [251]:
train_df.head()

,id,content,abstract,reverst_keys,high_key,textrank,own_key
0,0,【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把...,用户来电反映手机号卡收费问题，用户称现在怎么欠费167.65，我方经查询用户有10月欠费76...,"['167.65', '此号', '就别', '扣取', '没用']","['用户', '费用', '欠费', '我方', '使用']","['用户', '服务', '费用', '认可', '没用']",
1,1,【客户】嗯，【坐席】，很高兴为您服务，【客户】喂，你好，你好，我想问一下，我没有去过这个吉林...,用户来电反映查询漫游地，显示有吉林，用户称没有去过附近，要求核实回复，请处理谢谢,"['吉林', '附近', '漫游', '去过', '谢谢']","['用户', '查询', '核实', '反映', '没有']","['用户', '漫游', '查询', '核实', '回复']",
2,2,【客户】嗯，【坐席】，您好，实习话务员为您服务，【客户】喂你好，然后我这某某人卡，然后有有两...,用户来电想要取消20元语音特惠包（200分钟另得100条短信）-立即生效，我方取消不成功，用...,"['另得', '特惠', '200', '想要', '立即']","['用户', '我方', '取消', '核实', '短信']","['用户', '取消', '回复', '想要', '要求']",[短信]
3,3,【坐席】很高兴为您服务，您好？【客户】，唉，您好，那个，我这手机号上个月就是因为我每个月固定...,用户来电反映自己手机号码在7月份24.10元 8月份元 11月份85.90元 12月份300...,"['24.10', '85.90', '详细', '300', '11']","['用户', '月份', '我方', '认可', '流量']","['认可', '用户', '月份', '处理', '手机号码']",[流量]
4,4,【坐席】您好，高兴为您服务，【客户】，唉，你好那个，我家里这个***不太好，【坐席】信号不好...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,"['谢谢', '以前', '宝贵', '持续', '这些']","['我们', '信号', '最近', '使用', '问题']","['留意', '时间', '发生', '反馈', '问题']",[信号]


In [252]:
train_df[train_df['own_key']=='']

,id,content,abstract,reverst_keys,high_key,textrank,own_key
0,0,【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把...,用户来电反映手机号卡收费问题，用户称现在怎么欠费167.65，我方经查询用户有10月欠费76...,"['167.65', '此号', '就别', '扣取', '没用']","['用户', '费用', '欠费', '我方', '使用']","['用户', '服务', '费用', '认可', '没用']",
1,1,【客户】嗯，【坐席】，很高兴为您服务，【客户】喂，你好，你好，我想问一下，我没有去过这个吉林...,用户来电反映查询漫游地，显示有吉林，用户称没有去过附近，要求核实回复，请处理谢谢,"['吉林', '附近', '漫游', '去过', '谢谢']","['用户', '查询', '核实', '反映', '没有']","['用户', '漫游', '查询', '核实', '回复']",
5,5,【坐席】很高兴为您服务，您好？【客户】，唉，你是联通吗【坐席】嗯，对，这里是北京联通客服中心...,客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获...,"['点击', '错误', '重新', '通信', '中心']","['客户', '查询', '反映', '无法', '验证码']","['验证码', '获取', '反映', '查询', '点击']",
13,13,【坐席】您好，很高兴为您服务？【客户】，喂，你好，我就想咨询一下这个，我是18年买的这个纪念...,用户来电称2013年通过营业厅购买“纪念毛泽东诞辰120周年钞劵版连体电话卡”体现已过期，开...,"['纪念', '毛泽东', '诞辰', '连体', '序号']","['用户', '我方', '办理', '认可', '解释']","['用户', '体现', '充值', '购买', '纪念']",
17,17,【坐席】您好，很高兴为您服务，【客户】，嗯，你好，我这边遇到一个一个叫什么叫手机红叉的，这个...,******** 韦雪源 北京沃派校园套餐57元（优化版） 客户使用的是苹果手机，手机总是受...,"['灵山县', '烟墩', '凤山', '二队', '韦雪源']","['客户', '使用', '号码', '要求', '手机']","['客户', '对方', '手机', '电话', '使用']",
...,...,...,...,...,...,...,...
24986,24986,【坐席】您好高兴服务，喂，你好，你好【客户】，就是上次我发，【坐席】反映的那个我那个话费，关...,用户再次来电反映***TS********，用户称我方承诺为其核减，一直没有到账，要求我方核...,"['太平村', '南台', '十渡', '镇西', '谢谢']","['用户', '我方', '核实', '反映', '没有']","['用户', '要求', '核实', '原因', '镇西']",
24990,24990,【坐席】很高兴为您服务，您好？【客户】，唉，你好，嗯，你好，我这个套餐里面好像有一个那个爱奇...,用户来电要求取消套餐中的 话费购，未验证机主信息，已向用户正常解释，用户要求我方协助处理，烦...,"['验证', '工作人员', '话费', '协助', '谢谢']","['用户', '要求', '我方', '处理', '套餐']","['用户', '要求', '处理', '套餐', '烦请']",
24992,24992,【坐席】您好，很高兴为您服务，我的手机怎么突然就断网了呢？手机上不能上网，稍等，我看一下啊，...,用户对其号码“北京5G套餐预存200享7折优惠1年”业务在2020年11月份到期有异议。我方...,"['优惠期', '核对', '过程', '200', '异议']","['用户', '业务', '办理', '我方', '号码']","['业务', '用户', '办理', '北京', '解释']",
24995,24995,【坐席】您好，很高兴为您服务，你好，那个是这样的，之前我就投诉我的，一直没有给我反馈，我也没...,用户再次来电反映新安装了一个宽带********，赠送的主卡还没有激活，现在想把******...,"['主卡', '工单', '加副卡', '淘宝', '满足']","['用户', '使用', '认可', '告知', '办理']","['告知', '认可', '用户', '需要', '安装']",


In [256]:
train_df.to_csv('datasets/train_dataset_key.csv', index=False, sep='|')